In [3]:
"""Package Loading"""
"""General"""
from webscraping_dictionaries import *
from webscraping_driver import *
from webscraping_packages import *
from webscraping_functions import *
#from webscraping_functions import *
#from email.mime.text import MIMEText
#from email.mime.multipart import MIMEMultipart
"""Chrome Options"""
#from selenium.webdriver import Chrome 
#from selenium.webdriver.chrome.service import Service 
#from webdriver_manager.chrome import ChromeDriverManager

"""Firefox Options"""
#from selenium.webdriver import Firefox
#from selenium.webdriver.firefox.service import Service
#from webdriver_manager.firefox import GeckoDriverManager


"""Initialize the driver"""

"""Driver Setup"""

#following steps from here: https://www.zenrows.com/blog/scraping-javascript-rendered-web-pages#installing-the-requirements
#critical help using the click method from here https://www.geeksforgeeks.org/click-element-method-selenium-python/

"""Chrome Version"""
#options = webdriver.ChromeOptions()
#options.headless = True
#options.page_load_strategy = 'none' 
#chrome_path = ChromeDriverManager().install()
#chrome_service = Service(chrome_path)
#driver = Chrome(options=options, service=chrome_service) 
#driver.implicitly_wait(5)

"""Firefox Version"""  
#options = webdriver.FirefoxOptions()
#options.headless = True
#options.add_argument("--headless=new")
#options.page_load_strategy = 'none' 
#firefox_path = GeckoDriverManager().install() 
#firefox_service = Service(firefox_path)
#driver = Firefox(options=options, service=firefox_service)
#driver.implicitly_wait(5)

'Firefox Version'

"""DEBUGGING SANDBOX"""

In [ ]:
#table this for now, since we still want to check for comp plan and ordinance keywords anyway
"Charlotte County Solar Development Page"
def charlotte_county_solar_development():
    driver.get("https://www.charlottecountyva.gov/departments/planning___zoning/solar_development.php")
    time.sleep(1)
    table_rows = driver.find_elements(By.CSS_SELECTOR,"tr")
    

In [17]:
def search_pdf(meeting_links_list, locality_dictionary):
    messages=[]
    for item in meeting_links_list:
        driver.get(item)
        time.sleep(60)
        keywords = []
        #add specific page tags to locality dictionaries! Likely a lot of  the same, but good to continue to remove hardcoding
        pages = driver.find_elements(By.CSS_SELECTOR,"div[class*=page")
        for page in pages:
            driver.execute_script("arguments[0].scrollIntoView();", page)
            time.sleep(5)
            agenda_content = driver.find_elements(By.CSS_SELECTOR,agenda_content_tags[agendacenter_dictionary[locality_dictionary]['agenda_content']])
            readable = check_readability(agenda_content)
            if readable==True:
                agenda_search = search_agenda_for_keywords(agenda_content)
                if agenda_search != []:
                    keywords += agenda_search
            elif readable==False:
                messages.append("New meeting document available for " + agendacenter_dictionary[locality_dictionary]['name'] + ". Part of the document cannot be scanned for keywords. " + item)
        if keywords != []:
            unique_keywords = pd.Series(keywords).unique().tolist()
            messages.append("Keyword(s) " + ", ".join(unique_keywords) + " found in upcoming meeting for " + agendacenter_dictionary[locality_dictionary]['name'] + ". " + item)
    return pd.Series(messages).unique().tolist()

def search_webpage(meeting_links_list,locality_dictionary):
    messages=[]
    for item in meeting_links_list:
        driver.get(item)
        time.sleep(60)
        keywords = []
        agenda_content = driver.find_elements(By.CSS_SELECTOR,agenda_content_tags[agendacenter_dictionary[locality_dictionary]['agenda_content']])
        agenda_search = search_agenda_for_keywords(agenda_content)
        if agenda_search != []:
            keywords += agenda_search
        elif keywords != []:
            unique_keywords = pd.Series(keywords).unique().tolist()
            messages.append("Keyword(s) " + ", ".join(unique_keywords) + " found in upcoming meeting for " + agendacenter_dictionary[locality_dictionary]['name'] + ". " + item)
    return pd.Series(messages).unique().tolist()

def search_agenda_for_keywords(agenda_content):
    search_results = []
    for item in agenda_content:
        if 'Solar'in item.text or 'solar' in item.text:
            search_results.append('Solar')
        if 'Zoning Ordinance' in item.text or 'Zoning ordinance' in item.text or 'zoning ordinance' in item.text:
            search_results.append("Zoning Ordinance")
        if 'Comprehensive Plan' in item.text or 'Comprehensive plan' in item.text or 'comprehensive plan' in item.text:
            search_results.append("Comprehensive Plan")
    return pd.Series(search_results).unique().tolist()

"""AgendaCenter"""
def agendacenter(locality_dictionary):
    driver.get(agendacenter_dictionary[locality_dictionary]['url'])
    time.sleep(5)
    table_rows = driver.find_elements(By.CSS_SELECTOR, meetings_tags['agendacenter'])
    future_meetings = [item for item in table_rows if check_meeting_date(search_dates(item.text,languages=['en'])[0][0])==True]
    agenda_links = [item.find_elements(By.CSS_SELECTOR,"a")[1].get_attribute("href") for item in future_meetings]
    if agendacenter_dictionary[locality_dictionary]['agenda_type']=='pdf':
        messages = search_pdf(agenda_links,locality_dictionary)
    if agendacenter_dictionary[locality_dictionary]['agenda_type']=='webpage':
        messages = search_webpage(agenda_links,locality_dictionary)
    return messages

In [19]:
New_Alerts = []

"Run all the webscraping functions, single thread version"
for locality_dictionary in agendacenter_dictionary:
    try:
        alert = agendacenter(locality_dictionary)
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + agendacenter_dictionary[locality_dictionary]['name'] + " using AgendaCenter code."
        New_Alerts.append(error_alert)
        continue


In [20]:
New_Alerts

['Keyword(s) Solar, Comprehensive Plan, Zoning Ordinance found in upcoming meeting for Cumberland County. https://www.cumberlandcounty.virginia.gov/AgendaCenter/ViewFile/Agenda/_12302024-682',
 'New meeting document available for Town of Bedford. Part of the document cannot be scanned for keywords. https://www.bedfordva.gov/AgendaCenter/ViewFile/Agenda/_01022025-398',
 'Keyword(s) Zoning Ordinance, Comprehensive Plan found in upcoming meeting for City of Colonial Heights. https://www.colonialheightsva.gov/AgendaCenter/ViewFile/Agenda/_01072025-669',
 'New meeting document available for City of Petersburg. Part of the document cannot be scanned for keywords. http://www.petersburg-va.org/AgendaCenter/ViewFile/Agenda/_01022025-902, \n Keyword(s) Zoning Ordinance, Comprehensive Plan, Solar found in upcoming meeting for City of Petersburg. http://www.petersburg-va.org/AgendaCenter/ViewFile/Agenda/_01022025-902']

In [21]:
"""BoardDocs"""
def check_boarddocs_agendas(locality_dictionary):
    all_tabs=driver.find_elements(By.CSS_SELECTOR,"a[id*=ui-id-")
    meetings_tab = [item for item in all_tabs if item.text in ['MEETINGS','Meetings']]
    meetings_tab[0].click()
    time.sleep(10)
    #get all the meeting links
    all_meetings = driver.find_elements(By.CSS_SELECTOR, meetings_tags['boarddocs'])
    update_messages = []
    future_meetings = [item for item in all_meetings if search_dates(item.text,languages=['en'])!=None and check_meeting_date(search_dates(item.text,languages=['en'])[0][1].strftime('%m/%d/%Y'))==True ]
    for item in future_meetings:
        meeting_title = item.text
        #if the meeting hasn't happened yet, check for last minute agenda edits that might contain solar until it has passed
        item.click()
        time.sleep(10)
        #find the element to click to view the meeting agenda for this meeting
        #add a dictionary for accessing agenda links/buttons
        meeting_agenda = driver.find_element(By.CSS_SELECTOR, agenda_link_or_button_tags['boarddocs'])
        #click the meeting agenda button
        meeting_agenda.click()
        time.sleep(10)
        #pause, give the page time to load
        #Now to get ALL the agenda content
        all_agenda_topics = driver.find_elements(By.CSS_SELECTOR, agenda_content_tags['boarddocs'])
        #run the keyword search
        agenda_search = search_agenda_for_keywords(all_agenda_topics)
        if agenda_search !=[]:
            update_messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for " + locality_dictionary['name'] + " in " + meeting_title + ". " + locality_dictionary['url'])
        meetings_tab[0].click()
        time.sleep(10)
    return update_messages

def boarddocs(locality_dictionary):
    driver.get(boarddocs_dictionary[locality_dictionary]['url'])
    time.sleep(10)
    messages = check_boarddocs_agendas(locality_dictionary)
    if boarddocs_dictionary[locality_dictionary]['second_page']==True:
        govt_tab = driver.find_element(By.CSS_SELECTOR,"a[id*=btn-board")
        govt_tab.click()
        time.sleep(10)
        menu_options = driver.find_elements(By.CSS_SELECTOR,"a[class*=dropdown-item")
        for item in menu_options:
            if "Planning Commission" in item.text:
                planning_commission=item
        planning_commission.click()
        time.sleep(10)
        pc_messages = check_boarddocs_agendas(locality_dictionary)
        messages += pc_messages
    return messages

In [22]:

for locality_dictionary in boarddocs_dictionary:
    try:
        alert = boarddocs(locality_dictionary)
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + boarddocs_dictionary[locality_dictionary]['name'] + " using BoardDocs code."
        New_Alerts.append("\n" + error_alert)
        continue

In [23]:
New_Alerts

['Keyword(s) Solar, Comprehensive Plan, Zoning Ordinance found in upcoming meeting for Cumberland County. https://www.cumberlandcounty.virginia.gov/AgendaCenter/ViewFile/Agenda/_12302024-682',
 'New meeting document available for Town of Bedford. Part of the document cannot be scanned for keywords. https://www.bedfordva.gov/AgendaCenter/ViewFile/Agenda/_01022025-398',
 'Keyword(s) Zoning Ordinance, Comprehensive Plan found in upcoming meeting for City of Colonial Heights. https://www.colonialheightsva.gov/AgendaCenter/ViewFile/Agenda/_01072025-669',
 'New meeting document available for City of Petersburg. Part of the document cannot be scanned for keywords. http://www.petersburg-va.org/AgendaCenter/ViewFile/Agenda/_01022025-902, \n Keyword(s) Zoning Ordinance, Comprehensive Plan, Solar found in upcoming meeting for City of Petersburg. http://www.petersburg-va.org/AgendaCenter/ViewFile/Agenda/_01022025-902']

In [24]:
"CivicClerk"
def civicclerk(locality_dictionary):
    messages=[]
    driver.get(civicclerk_dictionary[locality_dictionary]['url'])
    time.sleep(10)
    all_meetings = driver.find_elements(By.CSS_SELECTOR,meetings_tags['civicclerk'])
    meetings_with_agendas = []
    for item in all_meetings:
        try:
            #test if there's a download button, indicating agenda files have been posted. Lack of files to scan will throw an error, and we won't waste time checking that meeting link for keywords
            #consider making this a dictionary of options as well
            item.find_element(By.CSS_SELECTOR,"button[id*=downloadFilesMenu")
            meetings_with_agendas.append(item)
        except:
            continue
    meeting_links = [item.find_element(By.CSS_SELECTOR,"a").get_attribute("href") for item in meetings_with_agendas]
    #this should be reworked and fit into the search_pdf function, with additional tag specification dictionaries
    for item in meeting_links:
        driver.get(item)
        time.sleep(10)
        #don't rely on this to always be the same, turn this into a dictionary of options, even if there's only one current option. It has changed in the past, it can change again.
        pdf_viewer_frame = driver.find_elements(By.CSS_SELECTOR,"iframe[id*=pdfViewerIframe")
        if pdf_viewer_frame != []:
            driver.switch_to.frame(pdf_viewer_frame[0])
            agenda_content = driver.find_elements(By.CSS_SELECTOR,agenda_content_tags[civicclerk_dictionary[locality_dictionary]['agenda_content']])
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for " + civicclerk_dictionary[locality_dictionary['name']] + ". " + item)
    return messages

In [25]:

for locality_dictionary in civicclerk_dictionary:
    try:
        alert=civicclerk(locality_dictionary)
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + civicclerk_dictionary[locality_dictionary]['name'] + " using CivicClerk code"
        New_Alerts.append(error_alert)
        continue
 

C:\Users\E\AppData\Roaming\Python\Python312\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname TH identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


In [ ]:
       
for item in civicweb_dictionary:
    try:
        alert=civicweb(civicweb_dictionary[item][0],civicweb_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using CivicWeb code"
        New_Alerts.append(error_alert)
        continue
        
for item in document_center_dictionary:
    try:
        alert=document_center(document_center_dictionary[item][0],document_center_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using Document Center code"
        New_Alerts.append(error_alert)
        continue

for item in escribe_dictionary:
    try:
        alert=escribe(escribe_dictionary[item][0],escribe_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using EScribe code"
        New_Alerts.append(error_alert)
        continue

for item in granicus_dictionary:
    try:
        alert = granicus(granicus_dictionary[item][0],granicus_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using Granicus code"
        New_Alerts.append(error_alert)
        continue
        
for item in granicus_2_dictionary:
    try:
        alert = granicus_version_2(granicus_2_dictionary[item][0],granicus_2_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item + " using Granicus Version 2 code"
        New_Alerts.append(error_alert)
        continue

for item in laserfiche_dictionary:
    try:
        alert=laserfiche(laserfiche_dictionary[item][0],laserfiche_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using LaserFiche code"
        New_Alerts.append(error_alert)
        continue

for item in legistar_dictionary:
    try:
        alert=legistar(legistar_dictionary[item][0],legistar_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using Legistar code"
        New_Alerts.append(error_alert)
        continue

for item in meetingstable_dictionary:
    try:
        alert=meetings_table(meetingstable_dictionary[item][0],meetingstable_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using MeetingsTable code"
        New_Alerts.append(error_alert)
        continue

for item in novusagenda_dictionary:
    try:
        alert=novusagenda(novusagenda_dictionary[item][0],novusagenda_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using NovusAGENDA code"
        New_Alerts.append(error_alert)
        continue

for item in onbase_dictionary:
    try:
        alert=onbase(onbase_dictionary[item][0],onbase_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using OnBase code"
        New_Alerts.append(error_alert)
        continue

for item in primegov_dictionary:
    try:
        alert=prime_gov(primegov_dictionary[item][0],primegov_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using PrimeGov code"
        New_Alerts.append(error_alert)
        continue

for item in php_table_dictionary:
    try:
        alert=php_table(php_table_dictionary[item][0],php_table_dictionary[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert="Error webscraping " + item + " using PHP Table code"
        New_Alerts.append(error_alert)
        continue

for item in county_dictionary_single_variable:
    try:
        alert=county_dictionary_single_variable[item][0](county_dictionary_single_variable[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in county_dictionary_double_variable:
    try:
        alert=county_dictionary_double_variable[item][0](county_dictionary_double_variable[item][1],county_dictionary_double_variable[item][2])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in city_dictionary_single_variable:
    try:
        alert=city_dictionary_single_variable[item][0](city_dictionary_single_variable[item][1])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

for item in city_dictionary_double_variable:
    try:
        alert=city_dictionary_double_variable[item][0](city_dictionary_double_variable[item][1],city_dictionary_double_variable[item][2])
        if alert != []:
            New_Alerts.append(", \n ".join(alert))
    except:
        error_alert = "Error webscraping " + item
        New_Alerts.append(error_alert)
        continue

In [3]:
for item in New_Alerts:
    print(item)

Error webscraping Botetourt County using AgendaCenter code.
Error webscraping Campbell County using AgendaCenter code.
Error webscraping Caroline County using AgendaCenter code.
Error webscraping Cumberland County using AgendaCenter code.
Error webscraping Dinwiddie County using AgendaCenter code.
Error webscraping Franklin County using AgendaCenter code.
Error webscraping Halifax County using AgendaCenter code.
Error webscraping Henry County using AgendaCenter code.
Error webscraping King George County using AgendaCenter code.
Error webscraping Madison County using AgendaCenter code.
Error webscraping Mecklenburg County using AgendaCenter code.
Error webscraping Middlesex County using AgendaCenter code.
Error webscraping Page County using AgendaCenter code.
Error webscraping Patrick County using AgendaCenter code.
Error webscraping Powhatan County using AgendaCenter code.
Error webscraping Rockbridge County using AgendaCenter code.
Error webscraping Rockingham County using AgendaCente

In [4]:
#empty list 
Solar_Alerts = []
Error_Alerts = []
Unreadable_File_Alerts = []
Other_Alerts = []

#loop that checks if the word in the meeting agenda is in the list of keywords...if that happens, append to empty list
for message in New_Alerts:
    if "Solar" in message:
        Solar_Alerts.append(message)
    elif "Error" in message or "Not Reachable" in message:
        Error_Alerts.append(message)
    elif "scanned" in message:
        Unreadable_File_Alerts.append(message)
    else:
        Other_Alerts.append(message)

# Convert categorized alerts to strings
solar_alerts_str = ", \n".join(Solar_Alerts)
error_alerts_str = ", \n".join(Error_Alerts)
unread_alerts_str = ", \n".join(Unreadable_File_Alerts)
other_alerts_str = ", \n".join(Other_Alerts)


In [5]:
alerts = ("Solar Alerts:\n" + solar_alerts_str + "\n\nUnreadable Alerts: \n" + unread_alerts_str + "\n\nError Alerts:\n" + error_alerts_str + "\n\nOther Alerts:\n" + other_alerts_str)

In [6]:
print(alerts)

Solar Alerts:
Keyword(s) Comprehensive Plan, Solar found in upcoming meeting for City of Suffolk. https://www.suffolkva.us/AgendaCenter/ViewFile/Agenda/_12172024-3425, 
 Keyword(s) Comprehensive Plan found in upcoming meeting for City of Suffolk. https://www.suffolkva.us/AgendaCenter/ViewFile/Agenda/_12172024-3426, 
Keyword(s) Solar found in upcoming meeting for Charles City County. https://charlescitycova.portal.civicclerk.com/event/162/files, 
 Keyword(s) Solar, Zoning Ordinance found in upcoming meeting for Charles City County. https://charlescitycova.portal.civicclerk.com/event/163/files, 
Keyword(s) Solar found in upcoming meeting for James City County. https://jamescitycova.portal.civicclerk.com/event/1301/files, 
Keyword(s) Zoning Ordinance found in upcoming meeting for Louisa County. https://louisacova.portal.civicclerk.com/event/971/files, 
 Keyword(s) Solar found in upcoming meeting for Louisa County. https://louisacova.portal.civicclerk.com/event/860/files, 
 Keyword(s) Sola

In [ ]:
#come back to this another day
all_dictionaries=[agendacenter_dictionary,boarddocs_dictionary,civicclerk_dictionary,civicweb_dictionary,document_center_dictionary,escribe_dictionary,granicus_dictionary,granicus_2_dictionary,laserfiche_dictionary,legistar_dictionary,meetingstable_dictionary,novusagenda_dictionary,php_table_dictionary,primegov_dictionary,county_dictionary_single_variable,county_dictionary_double_variable,city_dictionary_single_variable,city_dictionary_double_variable]

for dictionary in all_dictionaries:
    for entry in dictionary:
        print(entry)

In [ ]:
#reconfiguring how the scraping methods work
#leverage object oriented programming and the possibilities of functions
#currently, each website has its own custom function, and the repeated document sharing websites have their own customized function as well
#few repeated functions despite potential for more, and for better generalization
#inconsistent methodoloy that evolved over time as I kept remaking the functions from scratch
#potential to take the common steps and turn those into functions rather than having them rewritten dozens of times
#opportunities to keep optimizing

In [4]:
#test this later
"CivicClerk"
def civicclerk(locality_dictionary):
    messages=[]
    driver.get(civicclerk_dictionary[locality_dictionary]['url'])
    time.sleep(10)
    all_meetings = driver.find_elements(By.CSS_SELECTOR,meetings_tags['civicclerk'])
    meetings_with_agendas = []
    for item in all_meetings:
        try:
            #test if there's a download button, indicating agenda files have been posted. Lack of files to scan will throw an error, and we won't waste time checking that meeting link for keywords
            #consider making this a dictionary of options as well
            item.find_element(By.CSS_SELECTOR,"button[id*=downloadFilesMenu")
            meetings_with_agendas.append(item)
        except:
            continue
    meeting_links = [item.find_element(By.CSS_SELECTOR,"a").get_attribute("href") for item in meetings_with_agendas]
    #this should be reworked and fit into the search_pdf function, with additional tag specification dictionaries
    for item in meeting_links:
        driver.get(item)
        time.sleep(10)
        #don't rely on this to always be the same, turn this into a dictionary of options, even if there's only one current option. It has changed in the past, it can change again.
        pdf_viewer_frame = driver.find_elements(By.CSS_SELECTOR,"iframe[id*=pdfViewerIframe")
        if pdf_viewer_frame != []:
            driver.switch_to.frame(pdf_viewer_frame[0])
            agenda_content = driver.find_elements(By.CSS_SELECTOR,agenda_content_tags[civicclerk_dictionary[locality_dictionary]['agenda_content']])
            agenda_search = search_agenda_for_keywords(agenda_content)
            if agenda_search != []:
                messages.append("Keyword(s) " + ", ".join(agenda_search) + " found in upcoming meeting for " + civicclerk_dictionary[locality_dictionary['name']] + ". " + item)
    return messages

In [4]:
civicclerk('Charlottesville')

NameError: name 'driver' is not defined